In [1]:
{-# LANGUAGE TypeOperators, ParallelListComp, ScopedTypeVariables, Rank2Types, UnicodeSyntax #-}
:opt no-lint

In [2]:
import Data.VectorSpace
import Math.LinearMap.Category
import Prelude ()
import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained
import Data.Semigroup ((<>))
type ℝ = Double

In [3]:
import System.Random
import Control.Monad (replicateM)
randomPoints <- replicateM 100 $ randomRIO (0,10) :: IO [ℝ]
randomDeviations <- cycle <$> replicateM 165 `id` randomRIO (-0.1, 0.1) :: IO [ℝ]

In [4]:
mapM_ print (
 linearRegressionW euclideanNorm
    (\x -> lfun $ \(a,b) -> a*x + b)
 <$> [ [(0,1), (1,2), (2,3)]
     , [(-2,6), (0,2), (1,0)]
     , [(x, 14.3*x + 2.9 + δy) | x<-randomPoints
                               | δy<-randomDeviations]
     :: [(ℝ,ℝ)] ])

(1.0,1.0)
(-2.0,1.9999999999999996)
(14.30043273454546,2.902335291125837)

In [5]:
import Data.List (sort)
newtype Take = Take (∀ a . Int -> [a] -> [a])
takeSome' <- do
   randomIndices <- replicateM 10000 $ randomRIO (0, 10000)
   return $ Take (\n xs -> let l = length xs
                               go (i₀:i₁:is) xs'
                                | (e:es, r) <- splitAt (i₁-i₀) xs'  = e : go (i₁:is) r
                               go (i₀:is) (x₀:xs')  = x₀ : go is xs'
                               go _ _ = []
                           in go (sort . take n $ (`mod`l)<$>randomIndices) xs)
takeSome :: Int -> [a] -> [a]
Take takeSome = takeSome'

In [6]:
import Linear.V2 (V2(..), _x, _y)

mapM_ print (
 linearRegressionW euclideanNorm
    (\v -> lfun $ \((a,b),c) -> a<.>^squareV v + b<.>^v + c)
 <$> [ takeSome n [ (V2 x y, 3*x + 2*y + x*y + 4 + δy)
       | x<-randomPoints
       , y<-randomPoints
       | δy<-randomDeviations ]
     | n <- [5, 6, 10, 500]] )

((SymTensor {getSymmetricTensor = ex.⊗V2 1.4764217659831047e-2 0.20196931544951582 ^+^ ey.⊗V2 0.20196931544951582 0.524045255035162},V2 5.664306163787842 (-0.36726121604442596)),3.7917109355330467)
((SymTensor {getSymmetricTensor = ex.⊗V2 (-2.018501994387023e-2) 0.4863668100652247 ^+^ ey.⊗V2 0.4863668100652247 5.366703206368584e-2},V2 3.4032321264439815 1.5383411331108618),4.125791521379142)
((SymTensor {getSymmetricTensor = ex.⊗V2 8.487245695754098e-4 0.503252227259141 ^+^ ey.⊗V2 0.503252227259141 (-5.148376967220258e-3)},V2 2.9559152502942823 2.0267742239693263),4.071646522187848)
((SymTensor {getSymmetricTensor = ex.⊗V2 (-3.928365862648775e-5) 0.4999193604125227 ^+^ ey.⊗V2 0.4999193604125227 (-2.2165752794478522e-4)},V2 3.0008262090145195 2.0049998995953615),3.9833497589634135)

In [7]:
abc :: ((SymmetricTensor ℝ (V2 ℝ) +> ℝ, V2 ℝ +> ℝ), ℝ)
abc =
 linearRegressionW euclideanNorm
    (\v -> lfun $ \((a,b),c) -> (a $ squareV v) + (b $ v) + c)
  $ [ (V2 x y, 3*x + 2*y + x*y + 4 + δy)
       | x<-randomPoints
       , y<-randomPoints
       | δy<-randomDeviations ]
((a,b),c) = abc

In [8]:
import Control.Lens
import Math.LinearMap.Category.Derivatives

b ^. (1*∂id/∂_x)
b ^. (1*∂id/∂_y)

3.000578466315119

2.000676524590631

In [9]:
ðxx, ðxy :: Lens' (V2 ℝ⊗〃+>ℝ) ℝ
ðxx = (1*∂id/∂_x.∂_x)
ðxy = (1*∂id/∂_x.∂_y)
a ^. ðxx
a ^. ðxy

-4.955132155259889e-5

0.9999132161783449

In [10]:
(a & ðxx .~ 4) ^. ðxx
(a & ðxx .~ 4) ^. ðxy

4.0

0.9999132161783449

In [11]:
let n = euclideanNorm :: Norm ℝ
 in mapM_ (\(m,dvs) -> print (m, convexPolytopeRepresentatives dvs :: [(ℝ,ℝ)]))
     $ linearRegressionWVar (\x -> lfun $ \(a,b) -> a*x + b)
     <$> [ [(-1,(0,n)), (0,(-1,n)), (0,(1,n)), (1,(0,n))]
         , [( 0,(0,n)), (1,(-1,n)), (1,(1,n)), (2,(0,n))]
         , [( 0,(0,n)), (1,(-1,n)), (1,(3,n)), (2,(0,n))]
         , [( 0,(0,n)), (1,(-1,n)), (1,(3,scaleNorm (sqrt 3/3) n)), (2,(0,n))]
         , [( 0,(0,n)), (0,(2,n)), (1,(1,n)), (1,(3,n))] ]

((0.0,0.0),[(0.0,-1.0),(0.0,1.0)])
((0.0,0.0),[(2.1230172241752563e15,-2.1230172241752573e15),(-2.1230172241752563e15,2.1230172241752573e15)])
((0.0,0.5),[(-1.3433698597964394e-16,-1.4999999999999998),(1.8657914719394995e-16,2.5)])
((0.0,0.0),[(-1.0,0.0),(3.0,0.0)])
((1.0,1.0),[(0.9999999999999999,-1.0),(-0.9999999999999999,1.0),(-0.9999999999999999,-1.5700924586837752e-16),(0.9999999999999999,1.5700924586837752e-16)])

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [12]:
import Graphics.Dynamic.Plot.R2
import Data.Manifold.Shade ((|±|))

In [13]:
second convexPolytopeRepresentatives
 $ linearRegressionWVar (\x -> lfun $ \(c₀,(b₁,c₁)) -> c₀ + b₁*sin x + c₁*cos x)
                              (takeSome 30 [ (x, (sqrt 2 * sin (x + pi/4) + δy, spanNorm [1/0.05]))
                                           | x<-randomPoints
                                           | δy<-randomDeviations ])
    :: ((ℝ,(ℝ,ℝ)), [(ℝ,(ℝ,ℝ))])

((-5.971905352110368e-3,(1.0269988083972055,0.962078196908117)),[(3.0496595469200844e-3,(3.89100426740403e-2,7.278303181260681e-2)),(-5.638438573379918e-2,(2.9492609787295967e-2,5.682197433011512e-2)),(-1.9291689933220026e-2,(2.9250685220981884e-2,1.3383203630771751e-3)),(3.578427910961642e-2,(-5.416829778925377e-2,-2.8396649690092335e-3)),(3.617086182766489e-2,(-3.185491743365629e-2,-2.9542418215195352e-2)),(2.8204709053468696e-2,(1.4918467088033046e-2,-4.000439411307118e-2)),(6.780384916461815e-3,(-3.258004068130568e-2,-1.1780545428348173e-2)),(-3.775920519214847e-2,(-1.1724983702202285e-2,5.071193265645115e-2))])

In [14]:
plotWindow [linregressionPlot (\x -> lfun $ \(c₀,(b₁,c₁)) -> c₀ + b₁*sin x + c₁*cos x)
                              (takeSome 30 [ (x, sqrt 2 * sin (x + pi/4) + δy |±|[0.05])
                                           | x<-randomPoints
                                           | δy<-randomDeviations ])
                              (const (<>)) ]

GraphWindowSpecR2{lBound=-1.4955766206930592, rBound=11.500332964501087, bBound=-2.00021207819921, tBound=1.9652338755735261, xResolution=640, yResolution=480}

![Plot of regression of a set of deviated sine-sampled function points, with uncertainties.](https://raw.githubusercontent.com/leftaroundabout/linearmap-family/master/test/images/regression-envi.png)